# Keras
## Introducción
Keras es un API de alto nivel para redes neuronales, escrita en Python que es capaz de correr sobre TensorFlow, CNTK o Theano. Fue desarrollada con énfasis en habilitar experimentación rápida. Es recomendable el uso de Keras cuando se requiere de una libería de Deep Learning que:
- Permita el desarrollo de prototipos fácil y rápido (por medio de facilidad de uso, modularidad y extensibilidad).
- Soporte redes neuronales convolucionales (CNN) y redes neuronales recurrentes (RNN), así como combinaciones de ambas.
- Corra de forma natural en CPUs y GPUs.

Keras es compatible con Python 2.7-3.6.

[keras.io](https://keras.io) es el sitio principal del proyecto.

## Backend Keras y tf.keras
Se recomienda usar `tf.keras` a usuarios que usen Keras multi-backend con TensorFlow en TensorFlow 2.0 ya que `tf.keras` tiene mejor integración con las características de TensorFlow 2.0 (eager execution, soporte a distribución).

## Principios
- **Amigable**. El API está diseñado para personas, es consistente, reduce el número de pasos para casos de uso comunes. 
- **Modular**. Un modelo de deep learning consta de una secuencia o un grafo de módulos configurables que pueden ser conectados  con la menor cantidad de restricciones. Las capas de las redes neuronales, las funciones de costo, los optimizadores, esqueasm de inicialización, funciones de activación y esquemas de regularización son módulos independientes que pueden combinarse para crear nuevos modelos.
- **Extensible**. Es posible añadir módulos nuevos
- **Trabaja con Python**. No requiere de archivos de configuración de modelos. Los modelos se describen en Python, que es compacto, fácil de depurar y facilita la extensibilidad.

## Estructuras de datos
La estructura central de Keras es un modelo, que representa una forma de organizar capas. `Sequential` representa al modelo más simple, una pila linear de capas. El **Functional API**, por medio de `Model` permite crear arquitecturas más complejas por medio de un grafo arbitrario de capas.

## Clasificación de dígitos en Keras


Importar datos

In [9]:
import numpy as np
import mnist
import keras

from keras.utils import to_categorical


train_images = mnist.train_images()
train_labels = mnist.train_labels()

test_images = mnist.test_images()
test_labels = mnist.test_labels()

print(train_images.shape) # (60000, 28, 28)
print(train_labels.shape) # (60000,)

print(test_images.shape)
print(test_labels.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


### Preprocesar los datos

In [10]:
# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Flatten the images.
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

print(test_images.shape)
print(test_labels.shape)

(10000, 784)
(10000,)


### Construir el modelo

`model = Sequential([  #layers...])`

El constructor de Sequential recibe como parámetro un arreglo de [Layers](https://keras.io/layers/about-keras-layers/) de Keras

Para el caso de una red neuronal feed forward solo se requiere de la capa [Dense](https://keras.io/layers/core/#dense), que representa una capa completamente conectada.

En este ejemplo añadiremos tres capas al modelo, las primeras dos con 64 nodos cada unoa con la función de activación [relu](https://medium.com/@danqing/a-practical-guide-to-relu-b83ca804f1f7) y la última con 10 nodos y función de activación [softmax](https://victorzhou.com/blog/softmax/).

Keras requiere conocer la forma de los datos de entrada por medio del parámetro `input_shape` especificado en la primera capa.

In [13]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])

### Compilar el modelo
Es necesario configurar el proceso de entrenamiento considerando tres factores:
- El *optimizador*.
- La *función de pérdida (loss function)*.
- Las *métricas* de optimización.

In [14]:
model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

### Entrenar el modelo
Para entrenar el modelo se invoca la función `fit` con una serie de [parámetros](https://keras.io/models/sequential/#fit), entre los que destacan:
- *Datos de entrenamiento*, incluyendo datos y etiquetas.
- *Número de épocas (epochs)*, que representa el número de iteraciones sobre el dataset completo.
- *El tamaño del batch* número de muestras por actualización de gradiente.

Para este caso Keras espera los targets de entrenamiento como un vector de 10 dimensiones, dado que hay 10 nodos en la capa de salida Softmax, sin embargo train_labels provee de un solo entero que representa la clase de cada imagen. Es por ello que se hace uso del método [to_categorical](https://keras.io/utils/#to_categorical) que convierte al arreglo de enteros en un arreglo de vectores one-hot., por ejemplo 2 estaría representado como `[0,0,1,0,0,0,0,0,0,0,0]` (indexado en cero).

In [17]:
model.fit(
  train_images, # training data
  to_categorical(train_labels), # training targets
  epochs=5,
  batch_size=32,
)

Epoch 1/5
60000/60000 [==============================] - 15s 254us/step - loss: 0.3564 - accuracy: 0.8918
Epoch 2/5
60000/60000 [==============================] - 9s 145us/step - loss: 0.1830 - accuracy: 0.9444
Epoch 3/5
60000/60000 [==============================] - 10s 173us/step - loss: 0.1378 - accuracy: 0.9576
Epoch 4/5
60000/60000 [==============================] - 9s 152us/step - loss: 0.1170 - accuracy: 0.9638
Epoch 5/5
60000/60000 [==============================] - 9s 156us/step - loss: 0.1015 - accuracy: 0.9682


### Probar el modelo
Keras provee el método [evaluate](https://keras.io/models/sequential/#evaluate) que regresa un arreglo con la pérdida en prueba (test loss) seguida de las métricas especificadas.

In [18]:
model.evaluate(
  test_images,
  to_categorical(test_labels)
)

10000/10000 [==============================] - 1s 51us/step


[0.09909474250399508, 0.9684000015258789]

### Usar el modelo
El modelo puede guardarse y recuperarse de disco por medio de las funciones `save_weights` y `load_weights`

La función `predict` de un modelo permite predecir con datos de prueba o instancias nuevas.

In [19]:

# Predict on the first 5 test images.
predictions = model.predict(test_images[:5])

# Print our model's predictions.
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
print(test_labels[:5]) # [7, 2, 1, 0, 4]

[7 2 1 0 4]
[7 2 1 0 4]
